# Inicialização

In [3]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
import  psutil
import  os
import  time
import  joblib
import  warnings
import  pandas as pd
import  seaborn as sns
import  snap7
import  ctypes
import  numpy as np
import  matplotlib.pyplot as plt

from    torch.utils.data import DataLoader, TensorDataset
from    sklearn.model_selection import train_test_split
from    sklearn.metrics import confusion_matrix, classification_report
from    sklearn.ensemble import RandomForestClassifier
from    sklearn.preprocessing import LabelEncoder
from    sklearn.model_selection import GridSearchCV
from    sklearn.preprocessing import StandardScaler
from    sklearn.preprocessing import MinMaxScaler
from    sklearn.model_selection import ParameterGrid
#from   snap7.types import Areas, S7DataItem, S7WLWord, S7WLReal, S7WLTimer
from    snap7.type import S7DataItem, Area, WordLen
from    snap7.util import set_int, set_real, get_int, get_real, get_s5time
from    imblearn.over_sampling import SMOTE
from    imblearn.under_sampling import RandomUnderSampler
from    imblearn.pipeline import Pipeline
from    imblearn.combine import SMOTETomek

In [4]:

warnings.filterwarnings("ignore")

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

print(device)

cuda


# Carregar e Preparar Conjunto de Dados

In [5]:
# dados importados da plataforma KAGGLE - https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification
df = pd.read_csv("predictive_maintenance.csv")
df.head()


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


* descartar as colunas desnecessárias

In [6]:
df=df.drop(['Product ID','UDI','Target'],axis=1)

* codificar a coluna de falhas para tipo int entre 0 e 5

In [7]:
label_encoder = LabelEncoder()

# Perform label encoding
df['Failure Type_encoded'] = label_encoder.fit_transform(df['Failure Type'])

df['Failure Type_encoded'].value_counts()

''''
0 - 112  - heat dissipation
1 - 9652 - no Failure
2 - 78   - overstrain
3 - 95   - power failure
4 - 18   - random failure
5 - 45   - tool wear
'''


"'\n0 - 112  - heat dissipation\n1 - 9652 - no Failure\n2 - 78   - overstrain\n3 - 95   - power failure\n4 - 18   - random failure\n5 - 45   - tool wear\n"

* converter a coluna 'Type' (string) em outras duas separadas do tipo bool (uma para cada tipo de material) 

In [8]:
df = pd.get_dummies(df, columns=['Type'], prefix='Type', drop_first=True)
df.head()


,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type,Failure Type_encoded,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,No Failure,1,False,True
1,298.2,308.7,1408,46.3,3,No Failure,1,True,False
2,298.1,308.5,1498,49.4,5,No Failure,1,True,False
3,298.2,308.6,1433,39.5,7,No Failure,1,True,False
4,298.2,308.7,1408,40.0,9,No Failure,1,True,False


* separar os dados em: dados de entrada (features), X  e dados de saída (label), y

In [9]:
# Definição do conjunto de dados de entrada e do conjunto de dados de saída

X_raw = df.drop(['Failure Type','Failure Type_encoded'], axis=1).values
y_raw= df['Failure Type_encoded'].values

* Em aplicações do mundo real, a modelagem de classificação frequentemente enfrenta o problema de conjuntos de dados desequilibrados, onde o número de instâncias da classe majoritária é muito maior do que o da classe minoritária, o que dificulta o aprendizado adequado do modelo em relação à classe minoritária. Isso se torna um problema sério quando a informação contida na classe minoritária é mais importante como, por exemplo, no conjunto utilizado.

* Uma das abordagens populares para resolver o problema de conjuntos de dados desequilibrados é a superamostragem da classe minoritária ou a subamostragem da classe majoritária. No entanto, essas abordagens possuem suas próprias limitações. No método tradicional de superamostragem, a ideia é duplicar aleatoriamente alguns exemplos da classe minoritária — essa técnica não adiciona novas informações ao conjunto de dados. Por outro lado, o método de subamostragem é realizado removendo aleatoriamente alguns exemplos da classe majoritária, o que resulta na perda de algumas informações originais dos dados.

In [10]:
print('Total de amostras antes balanceamento:',df['Failure Type_encoded'].count())
print('Total de amostras por falha:\n\n', df['Failure Type_encoded'].value_counts())

Total de amostras antes balanceamento: 10000
Total de amostras por falha:

 Failure Type_encoded
1    9652
0     112
3      95
2      78
5      45
4      18
Name: count, dtype: int64


* Uma solução para superar essas limitações é gerar novos exemplos sintetizados a partir da classe minoritária existente. Esse método é conhecido como Técnica de Superamostragem da Minoria Sintética (SMOTE). Existem muitas variações do SMOTE, mas neste artigo será explicado o método SMOTE-Tomek Links e sua implementação em Python. Esse método combina a superamostragem do SMOTE com a subamostragem dos Tomek Links.

* O método SMOTE-Tomek Links, desenvolvido por Chawla et al. (2002) é uma combinação de técnicas que visa equilibrar os dados ao aumentar a representatividade da classe minoritária através da criação de novos exemplos sintéticos e ao mesmo tempo remover exemplos que são considerados ruidosos ou redundantes na classe majoritária. A implementação desta abordagem utilizando Python é apresentada, detalhando os passos e a lógica do algoritmo, assim como os resultados obtidos em diferentes conjuntos de dados desequilibrados.

* A combinação das técnicas de superamostragem e subamostragem, especificamente utilizando o método SMOTE-Tomek Links, mostra-se eficaz na melhoria do desempenho dos modelos de classificação em conjuntos de dados desequilibrados. Esta abordagem permite ao modelo aprender de forma mais eficiente a partir da classe minoritária, mantendo a integridade e a diversidade da informação no conjunto de dados.



In [11]:
#implementação do método SMOTE no conunto de dados
smt=SMOTETomek(sampling_strategy='auto',random_state=42)
X_resampled, y_resampled = smt.fit_resample(X_raw, y_raw)

# Convertendo para dataframe do pandas
df_resampled = pd.DataFrame(X_resampled, columns=[f'feature_{i}' for i in range(X_resampled.shape[1])])
df_resampled['Type_encoded'] = y_resampled 

df_resampled.head()

#distribuição das classes após balanceamento
print('Total de amostras após balanceamento:',df_resampled['Type_encoded'].count())
print('Total de amostras por falha:\n\n', df_resampled['Type_encoded'].value_counts())

Total de amostras após balanceamento: 57868
Total de amostras por falha:

 Type_encoded
2    9652
0    9649
5    9647
3    9645
4    9645
1    9630
Name: count, dtype: int64


In [12]:
df_resampled = df_resampled.rename(columns={'feature_0':'Temp_Ar'})
df_resampled = df_resampled.rename(columns={'feature_1':'Temp_Pr'})
df_resampled = df_resampled.rename(columns={'feature_2':'Vel_Spindle'})
df_resampled = df_resampled.rename(columns={'feature_3':'Torque'})
df_resampled = df_resampled.rename(columns={'feature_4':'Desg_Ferr'})
df_resampled = df_resampled.rename(columns={'feature_5':'Mat_L'})
df_resampled = df_resampled.rename(columns={'feature_6':'Mat_M'})

df_resampled = df_resampled.rename(columns={'Type_encoded':'Tipo_Falha'})

df_resampled.head()


,Temp_Ar,Temp_Pr,Vel_Spindle,Torque,Desg_Ferr,Mat_L,Mat_M,Tipo_Falha
0,298.1,308.6,1551.0,42.8,0.0,0.0,1.0,1
1,298.2,308.7,1408.0,46.3,3.0,1.0,0.0,1
2,298.1,308.5,1498.0,49.4,5.0,1.0,0.0,1
3,298.2,308.6,1433.0,39.5,7.0,1.0,0.0,1
4,298.2,308.7,1408.0,40.0,9.0,1.0,0.0,1


* Metodo para normalização do conjunto de dados e formatação para as diferentes redes neurais (MLP e KAN)

In [ ]:
def load_Preditive_Maintenance_Dataset():

    X_ = df_resampled.drop(['Tipo_Falha'], axis="columns")

    scaler = MinMaxScaler(feature_range=(-1, 1))
    X_   = scaler.fit_transform(X_)    
    joblib.dump(scaler, 'minmax_scaler.pkl')    

    y_ = df_resampled['Tipo_Falha']

    #convertendo os dados em tensores e atribuindo a sua excucção na CPU ou GPU (device) conforme disponibilidade avaliada no inicio do programa
    X = torch.tensor(X_, dtype = torch.float32).to(device)
    y = torch.tensor(y_.values, dtype = torch.long).to(device)  

    #separa os dados entre conjunto de dados para treinamento (80%) e conjunto de dados para teste/validação (20% <-> teste_size=0.2)
    train_data_, test_data_, train_target_, test_target_ = train_test_split(X, y, test_size=0.2, random_state=42)

    # geração de  mini batches para melhorar desempenho do treinamento
    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_data_, train_target_), batch_size=10, shuffle=True)   
    validation_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_data_, test_target_), batch_size=10, shuffle=False)  

    #conjunto de dados no formato a ser utilizado no redes MLP
    train_data_loader   = train_loader
    test_data_loader    = validation_loader

    train_data      = train_data_    
    test_data       = test_data_
    train_labels    = train_target_
    test_label      = test_target_

    MLP_dataset     = [train_data, train_labels, test_data, test_label, train_data_loader, test_data_loader ]  

    return MLP_dataset

dataset =load_Preditive_Maintenance_Dataset()

# Implementação Rede Neural Multilayer Perceptron

In [40]:

class MLP_Model(nn.Module):  
    def __init__(self, layer_sizes, dropout_prob=0.5):
        super(MLP_Model, self).__init__()
        self.layers = nn.ModuleList()
        self.dropouts = nn.ModuleList()
        
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))
            if i < len(layer_sizes) - 2: 
                self.dropouts.append(nn.Dropout(p=dropout_prob))
        
    def forward(self, x):
        for i in range(len(self.layers) - 1):
            x = F.relu(self.layers[i](x))
            x = self.dropouts[i](x)
        x = self.layers[-1](x)
        return x

def initialize_weights(model):
    for layer in model.layers:
        if isinstance(layer, nn.Linear):
            nn.init.kaiming_normal_(layer.weight)
            if layer.bias is not None:
                nn.init.zeros_(layer.bias)

def train(model, train_loader, criterion, optimizer, num_epochs, device):
    model.train()

    ram_usage       = []
    cpu_usage       = []
    backprop_time   = []
    loss_list       = []

    startMLP_train = time.time()

    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            startMLP_bwd = time.time()
            loss.backward()
            optimizer.step()
            endMLP_bwd = time.time()
            backprop_time.append((endMLP_bwd - startMLP_bwd) * 1000)

        loss_list.append(loss.item())
        ram_usage.append(psutil.virtual_memory().percent)
        cpu_usage.append(psutil.cpu_percent(interval=1))
        print(f'Epoch {epoch+1}/{num_epochs} | Loss: {round(loss.item(), 4)} | Backpropagation Time: {round(backprop_time[-1], 2)} ms | CPU Usage: {round(cpu_usage[-1], 2)} % | RAM Usage: {round(ram_usage[-1], 2)} %', end="\r")

    endMLP_train = time.time()
   
    train_time = endMLP_train - startMLP_train
    backpropagation_time = np.mean(backprop_time)
    max_cpu_usage = np.max(cpu_usage)   
    max_ram_usage = np.max(ram_usage)

    info = {
        'Num_Epochs': num_epochs,
        'Back_Propagation_Time': backpropagation_time,
        'Max_CPU_usage': max_cpu_usage,
        'Max_RAM_usage': max_ram_usage,
        'Train_Time': train_time,
        'Loss_List': loss_list     }

    return info

def test_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

def run_grid_search(train_data, train_labels, test_data, test_labels, param_grid, num_epochs, device):
    results = []

    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_data, train_labels), batch_size=64, shuffle=True)   
    test_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_data, test_labels), batch_size=64, shuffle=False)  
    
    for params in ParameterGrid(param_grid):

        print('Configuração em execução: ',params)

        layer_sizes = [7] + params['hidden_layers'] + [6]
        model = MLP_Model(layer_sizes, dropout_prob=params['dropout_prob']).to(device)
        initialize_weights(model)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'], weight_decay=params['weight_decay'])
        train_results = train(model, train_loader, criterion, optimizer, params['epochs'], device)
        accuracy = test_model(model, test_loader, device)
        result = {
            'params': params,
            'num_parameters': sum(p.numel() for p in model.parameters() if p.requires_grad),
            'accuracy': accuracy,
            'loss_list': train_results['Loss_List'],
            'training_time' :  train_results['Train_Time'],
            'backpropagation_time' : train_results['Back_Propagation_Time'],
            'max_cpu_usage': train_results['Max_CPU_usage'],
            'max_ram_usage': train_results['Max_RAM_usage']
        }

        results.append(result)
    
    return results


In [41]:

# Definição dos hiperparâmetros para a busca em grade
param_grid = {
    'hidden_layers': [[10], [20, 10], [10, 10, 10], [50, 50, 50], [128, 64, 32], [128,64,32,16]],
    'learning_rate': [0.01, 0.005, 0.001, 0.0001],
    'weight_decay' : [0.0, 0.0001, 0.001],  # Regularização L2
    'dropout_prob' : [0.0, 0.3, 0.5],  # Probabilidade de dropout
    'epochs'       : [10, 50, 100, 200, 350]
    }

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_data      = dataset[0]
train_labels    = dataset[1]
test_data       = dataset[2]
test_labels     = dataset[3]
num_epochs       = 1

# Executar a busca em grade
results = run_grid_search(train_data, train_labels, test_data, test_labels, param_grid, num_epochs, device)

# Armazenar os resultados em um DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv('grid_search_results.csv', index=False)

# Exibir os melhores resultados
best_result = max(results, key=lambda x: x['accuracy'])

print(f"Number of Parameters: {best_result['num_parameters']}")
print(f"Backpropagation Time: {best_result['backpropagation_time']}%")
print(f"Max CPU Usage: {best_result['max_cpu_usage']}%")
print(f"Max RAM Usage Time: {best_result['max_ram_usage']}%")
print(f"Training Time: {best_result['training_time']}%")
print(f"Accuracy: {best_result['accuracy']}%")
print(f"\nMelhor configuração: {best_result['params']}")


Configuração em execução:  {'dropout_prob': 0.0, 'epochs': 10, 'hidden_layers': [10], 'learning_rate': 0.01, 'weight_decay': 0.0}
Configuração em execução:  {'dropout_prob': 0.0, 'epochs': 10, 'hidden_layers': [10], 'learning_rate': 0.01, 'weight_decay': 0.0001}
Configuração em execução:  {'dropout_prob': 0.0, 'epochs': 10, 'hidden_layers': [10], 'learning_rate': 0.01, 'weight_decay': 0.001}
Configuração em execução:  {'dropout_prob': 0.0, 'epochs': 10, 'hidden_layers': [10], 'learning_rate': 0.005, 'weight_decay': 0.0}
Configuração em execução:  {'dropout_prob': 0.0, 'epochs': 10, 'hidden_layers': [10], 'learning_rate': 0.005, 'weight_decay': 0.0001}
Configuração em execução:  {'dropout_prob': 0.0, 'epochs': 10, 'hidden_layers': [10], 'learning_rate': 0.005, 'weight_decay': 0.001}
Configuração em execução:  {'dropout_prob': 0.0, 'epochs': 10, 'hidden_layers': [10], 'learning_rate': 0.001, 'weight_decay': 0.0}
Configuração em execução:  {'dropout_prob': 0.0, 'epochs': 10, 'hidden_layer

KeyboardInterrupt: 

In [ ]:
MLP_loss_list = np.array([x.item() for x in mlp_train_results['Loss_List']]) 
iteration =  range(1, mlp_train_results['Num_Epochs']+1)        

plt.figure(figsize=(10, 6))
plt.plot(iteration, MLP_loss_list, marker='o', linestyle='-',  color='b', label='Erro Treino 1')

plt.title('Evolução do Ajuste do Erro')
plt.xlabel('Iterações')
plt.ylabel('Erro')
plt.grid(True)
plt.show()

INFERÊNCIA

In [ ]:
X_2d= X_raw[2581].reshape(1, -1)

print(X_2d)
print(y_raw[2581])

scaler = joblib.load('minmax_scaler.pkl')

# Aplicar a normalização aos novos dados
X_new_scaled = scaler.transform(X_2d)

# Converter para tensor PyTorch
X_new_scaled_tensor = torch.tensor(X_new_scaled, dtype=torch.float32)


In [48]:
def read_real_values(client, db_number, start_address, number_values):

    values = []

    try:
        # Conectando ao PLC
        #client.connect(plc_ip, 0, 1)

            # Lendo  valores reais (floats) do DB
        data = client.db_read(db_number, start_address, number_values*4)  # 4 bytes para cada valor real

            # Convertendo os bytes em floats
        values = [snap7.util.get_real(data, i) for i in range(0, number_values*4, 4)]

    except Snap7Exception as e:
        print(f"Erro ao ler dados do PLC: {e}")    
        
    return values

In [49]:

def write_data_to_plc(client, dbnumber,offset, value):

    realValue = bytearray(4)
    
    set_real(realValue, 0,value)

    client.db_write(dbnumber,offset,realValue)



In [50]:
plc_ip = '192.168.0.1'
db_number = 2
start_address = 0
read_number_of_values = 7

In [52]:
s71511T = snap7.client.Client()
s71511T.connect(plc_ip, 0, 1)

if  s71511T.get_connected() == True:
    
    while True:
        values=read_real_values(s71511T,2,0,7)

        features = np.array(values)
        features = features.reshape(1,7)

        dframe = pd.DataFrame(features)

        with torch.no_grad():
            output = model(dframe)

        write_data_to_plc(s71511T,3,0,predicted[0][0])
        write_data_to_plc(s71511T,3,4,predicted[0][1])
        write_data_to_plc(s71511T,3,8,predicted[0][2])
        write_data_to_plc(s71511T,3,12,predicted[0][3])
        write_data_to_plc(s71511T,3,16,predicted[0][4])
        write_data_to_plc(s71511T,3,20,predicted[0][5])
else:
    print("Sem conexão com o CLP")

TypeError: linear(): argument 'input' (position 1) must be Tensor, not DataFrame